In [2]:
from tensorflow import keras
from PIL import Image, ImageFilter, ImageChops

import numpy as np
import os

PATH_GOOD = "/home/air/Documentos/Projects/AIPhoto/crops_good/"
PATH_BAD = "/home/air/Documentos/Projects/AIPhoto/crops_bad/"

RENOIR_BAD = "/home/air/Documentos/Projects/AIPhoto/renoir/Mi3_Aligned/Batch_001/"

SIZE_INPUT = 128

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def build_model():
    model = keras.Sequential()
    
    model.add(keras.layers.Conv2D(kernel_size=(3,3), input_shape=(128,128,3), 
                                  filters=3, activation="relu", padding='same'))
    
    model.add(keras.layers.Conv2D(kernel_size=(2,2), filters=32, activation="relu", 
                                  padding='same', kernel_regularizer=keras.regularizers.l2(0.01)))
    
    model.add(keras.layers.Conv2D(kernel_size=(3,3), filters=64, activation="relu", 
                                 padding='same', kernel_regularizer=keras.regularizers.l2(0.02)))
    
    model.add(keras.layers.Conv2D(kernel_size=(1,1), filters=64, activation="relu", 
                                 padding='same'))
    
    model.add(keras.layers.Conv2D(kernel_size=(1,1), filters=128, activation="relu", 
                                 padding='same'))
     
    model.add(keras.layers.Conv2DTranspose(kernel_size=(2,2), filters=64, activation="relu", 
                                 padding='same'))
      
    model.add(keras.layers.Conv2DTranspose(kernel_size=(3,3), filters=64, activation="relu", 
                                 padding='same', 
                                           kernel_regularizer=keras.regularizers.l2(0.01)))
    
    model.add(keras.layers.Conv2DTranspose(kernel_size=(2,2), filters=32, activation="relu", 
                                 padding='same'))
    
    
    model.add(keras.layers.Conv2DTranspose(kernel_size=(3,3), filters=3, activation="relu", 
                                 padding='same'))
        
    model.compile(keras.optimizers.Adamax(), 
                  loss='mae')
  
    return model

In [4]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 3)       84        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      416       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 64)      4160      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 128)     8320      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 128, 128, 64)      32832     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 128, 128, 64)      36928     
__________

In [4]:
def load_dataset():
    array_good =[] 
    array_bad = [] 
    
    good_photos = sorted(os.listdir(PATH_GOOD)) # Nombres de las fotos
    bad_photos = sorted(os.listdir(PATH_BAD)) 
    
    #Cargar en batches para que quepan en memoria (PC poco potente)
    low = 150000
    high = low + 30000
    
    good_photos = good_photos[low:high]
    bad_photos = bad_photos[low:high]
    
    
    total = 0
    size = len(good_photos)
    
    for name in good_photos:
        image_good = Image.open(PATH_GOOD+name)
        image_bad = Image.open(PATH_BAD+name)
        
        array_good.append(np.array(image_good))
        array_bad.append(np.array(image_bad))
        
        total +=1
        print("Loaded {} of {}".format(total, size))
    
    return (array_good, array_bad)
        

In [5]:
def load_image(path):
    image = Image.open(path)
    arr_boxes = []
    size_x = image.width
    size_y = image.height
    
    batches_width = size_x // SIZE_INPUT
    batches_height = size_y // SIZE_INPUT
    
    print("{}x{}".format(batches_height, batches_width))
        
    for i in range(128, size_y, SIZE_INPUT):
        for j in range(128, size_x, SIZE_INPUT):
            crop_good = image.crop((j-128, i-128, j, i))
            arr_boxes.append(np.array(crop_good))
            
            #batches_width += 1
        #batches_height += 1
    return (arr_boxes, batches_width, batches_height)

In [6]:
""" 
    Hace una imagen a partir de una lista de ndarrays.
    width y height son el numero de bloques en los que se dividio la imagen original
""" 
def make_image(array, width, height):
    newIm = Image.new('RGB', (128*width, 128*height), color=(0,0,0))
    
    for i in range(height):
        for j in range(width):
            im = Image.fromarray(array[i*width+j].astype('uint8'))
            newIm.paste(im, (j*128,i*128))
    
    return newIm
    

In [7]:
def train_model(train_data, valid_data, model):
    model.fit(train_data, valid_data, epochs=4000, validation_split=0.3, batch_size=32, 
              metrics=['acc', 'mse'],
             callbacks=[keras.callbacks.EarlyStopping(patience=5),
                       keras.callbacks.TensorBoard(log_dir='tf_logs', histogram_freq=0, batch_size=32, write_graph=True, write_images=True)])

In [7]:
def predict(model, array_boxes):
    return model.predict(array_boxes)

In [8]:
def numpy_to_list(tensor_list):
    lista = []
    for tensor in tensor_list:
        lista.append(tensor)
    return lista

In [9]:
model = build_model()

In [11]:
data = load_dataset()

Loaded 1 of 30000
Loaded 2 of 30000
Loaded 3 of 30000
Loaded 4 of 30000
Loaded 5 of 30000
Loaded 6 of 30000
Loaded 7 of 30000
Loaded 8 of 30000
Loaded 9 of 30000
Loaded 10 of 30000
Loaded 11 of 30000
Loaded 12 of 30000
Loaded 13 of 30000
Loaded 14 of 30000
Loaded 15 of 30000
Loaded 16 of 30000
Loaded 17 of 30000
Loaded 18 of 30000
Loaded 19 of 30000
Loaded 20 of 30000
Loaded 21 of 30000
Loaded 22 of 30000
Loaded 23 of 30000
Loaded 24 of 30000
Loaded 25 of 30000
Loaded 26 of 30000
Loaded 27 of 30000
Loaded 28 of 30000
Loaded 29 of 30000
Loaded 30 of 30000
Loaded 31 of 30000
Loaded 32 of 30000
Loaded 33 of 30000
Loaded 34 of 30000
Loaded 35 of 30000
Loaded 36 of 30000
Loaded 37 of 30000
Loaded 38 of 30000
Loaded 39 of 30000
Loaded 40 of 30000
Loaded 41 of 30000
Loaded 42 of 30000
Loaded 43 of 30000
Loaded 44 of 30000
Loaded 45 of 30000
Loaded 46 of 30000
Loaded 47 of 30000
Loaded 48 of 30000
Loaded 49 of 30000
Loaded 50 of 30000
Loaded 51 of 30000
Loaded 52 of 30000
Loaded 53 of 30000
Lo

Loaded 442 of 30000
Loaded 443 of 30000
Loaded 444 of 30000
Loaded 445 of 30000
Loaded 446 of 30000
Loaded 447 of 30000
Loaded 448 of 30000
Loaded 449 of 30000
Loaded 450 of 30000
Loaded 451 of 30000
Loaded 452 of 30000
Loaded 453 of 30000
Loaded 454 of 30000
Loaded 455 of 30000
Loaded 456 of 30000
Loaded 457 of 30000
Loaded 458 of 30000
Loaded 459 of 30000
Loaded 460 of 30000
Loaded 461 of 30000
Loaded 462 of 30000
Loaded 463 of 30000
Loaded 464 of 30000
Loaded 465 of 30000
Loaded 466 of 30000
Loaded 467 of 30000
Loaded 468 of 30000
Loaded 469 of 30000
Loaded 470 of 30000
Loaded 471 of 30000
Loaded 472 of 30000
Loaded 473 of 30000
Loaded 474 of 30000
Loaded 475 of 30000
Loaded 476 of 30000
Loaded 477 of 30000
Loaded 478 of 30000
Loaded 479 of 30000
Loaded 480 of 30000
Loaded 481 of 30000
Loaded 482 of 30000
Loaded 483 of 30000
Loaded 484 of 30000
Loaded 485 of 30000
Loaded 486 of 30000
Loaded 487 of 30000
Loaded 488 of 30000
Loaded 489 of 30000
Loaded 490 of 30000
Loaded 491 of 30000


Loaded 969 of 30000
Loaded 970 of 30000
Loaded 971 of 30000
Loaded 972 of 30000
Loaded 973 of 30000
Loaded 974 of 30000
Loaded 975 of 30000
Loaded 976 of 30000
Loaded 977 of 30000
Loaded 978 of 30000
Loaded 979 of 30000
Loaded 980 of 30000
Loaded 981 of 30000
Loaded 982 of 30000
Loaded 983 of 30000
Loaded 984 of 30000
Loaded 985 of 30000
Loaded 986 of 30000
Loaded 987 of 30000
Loaded 988 of 30000
Loaded 989 of 30000
Loaded 990 of 30000
Loaded 991 of 30000
Loaded 992 of 30000
Loaded 993 of 30000
Loaded 994 of 30000
Loaded 995 of 30000
Loaded 996 of 30000
Loaded 997 of 30000
Loaded 998 of 30000
Loaded 999 of 30000
Loaded 1000 of 30000
Loaded 1001 of 30000
Loaded 1002 of 30000
Loaded 1003 of 30000
Loaded 1004 of 30000
Loaded 1005 of 30000
Loaded 1006 of 30000
Loaded 1007 of 30000
Loaded 1008 of 30000
Loaded 1009 of 30000
Loaded 1010 of 30000
Loaded 1011 of 30000
Loaded 1012 of 30000
Loaded 1013 of 30000
Loaded 1014 of 30000
Loaded 1015 of 30000
Loaded 1016 of 30000
Loaded 1017 of 30000
Lo

Loaded 1412 of 30000
Loaded 1413 of 30000
Loaded 1414 of 30000
Loaded 1415 of 30000
Loaded 1416 of 30000
Loaded 1417 of 30000
Loaded 1418 of 30000
Loaded 1419 of 30000
Loaded 1420 of 30000
Loaded 1421 of 30000
Loaded 1422 of 30000
Loaded 1423 of 30000
Loaded 1424 of 30000
Loaded 1425 of 30000
Loaded 1426 of 30000
Loaded 1427 of 30000
Loaded 1428 of 30000
Loaded 1429 of 30000
Loaded 1430 of 30000
Loaded 1431 of 30000
Loaded 1432 of 30000
Loaded 1433 of 30000
Loaded 1434 of 30000
Loaded 1435 of 30000
Loaded 1436 of 30000
Loaded 1437 of 30000
Loaded 1438 of 30000
Loaded 1439 of 30000
Loaded 1440 of 30000
Loaded 1441 of 30000
Loaded 1442 of 30000
Loaded 1443 of 30000
Loaded 1444 of 30000
Loaded 1445 of 30000
Loaded 1446 of 30000
Loaded 1447 of 30000
Loaded 1448 of 30000
Loaded 1449 of 30000
Loaded 1450 of 30000
Loaded 1451 of 30000
Loaded 1452 of 30000
Loaded 1453 of 30000
Loaded 1454 of 30000
Loaded 1455 of 30000
Loaded 1456 of 30000
Loaded 1457 of 30000
Loaded 1458 of 30000
Loaded 1459 o

KeyboardInterrupt: 

In [33]:
train_model(np.array(data[1]), np.array(data[0]), model)

Train on 21000 samples, validate on 9000 samples
Epoch 1/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5501 - val_loss: 5.9434
Epoch 2/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5991 - val_loss: 5.9404
Epoch 3/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5555 - val_loss: 6.2471
Epoch 4/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5725 - val_loss: 5.9387
Epoch 5/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5461 - val_loss: 5.9425
Epoch 6/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5668 - val_loss: 5.8328
Epoch 7/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5307 - val_loss: 6.0831
Epoch 8/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.5455 - val_loss: 5.9065
Epoch 9/4000
21000/21000 [==============================] - 169s 8ms/step - loss: 5.539

In [51]:
model.save_weights('v3-percent.h5')

In [10]:
model.load_weights('v3-percent.h5')

In [11]:
#Ejemplo de imagen
#Cambiar la ruta para proba otra.
ase = load_image('/home/air/Descargas/WhatsApp Image 2018-07-09 at 09.36.12.jpeg')

7x12


In [12]:
im = make_image(ase[0],ase[1],ase[2])

In [13]:
im.show()

In [14]:
im2_pred = predict(model, np.array(ase[0]))

In [15]:
#Normalizar si algo se pasa de 255 o de 0
im2_pred = np.clip(im2_pred, 0,255)


In [16]:
#Mostrar imagen
are = numpy_to_list(im2_pred)
make_image(are , ase[1], ase[2]).show()